# Data Science I - Projeto final

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.read_csv('titanic-data-6.csv')

# Preparar dados (Wrangle data)

Consultar o formato do dataframe, número de linhas e colunas.

In [ ]:
df.shape

Consultar as primeiro cinco linhas do dataframe.

In [ ]:
df.head()

Consultar informações como número de entradas, colunas e seus tipos de dados.

In [ ]:
df.info()

Renomear as colunas

In [ ]:
df.rename(
    columns={
        'PassengerId': 'id_passageiro', 
        'Survived': 'sobreviveu', 
        'Pclass':'classe', 
        'Name':'nome', 
        'Sex':'sexo', 
        'Age':'idade', 
        'SibSp':'irmao_conjuge', 
        'Parch':'pais_filhos', 
        'Ticket':'ticket', 
        'Fare':'tarifa', 
        'Cabin':'cabine',
        'Embarked': 'embarcou'
    }, inplace=True)

Verificar a existência de linhas duplicadas.

In [ ]:
df.duplicated().sum()

Verificar se há nomes repetidos

In [ ]:
df['nome'].duplicated().sum()

Verificar variedade de registros únicos em gênero (Sex)

In [ ]:
df['sexo'].nunique()

In [ ]:
df['sexo'].unique()

Tradução de valores de 'sexo'

In [ ]:
pt_sex = {'male': 'Masculino', 'female':'Feminino'}
df['sexo'] = df['sexo'].replace(pt_sex.keys(), pt_sex.values())

In [ ]:
df['sexo'].unique()

Verificar variedade de registros únicos em sobreviventes

In [ ]:
df['sobreviveu'].nunique()

In [ ]:
df['sobreviveu'].unique()

Substituição dos valores de sobreviventes de 0, 1 para 'Não' e 'Sim', respectivamente

In [ ]:
pt_survived = {'0':'Não', '1':'Sim'}
df['sobreviveu'] = df['sobreviveu'].astype(str).replace(pt_survived.keys(), pt_survived.values())

In [ ]:
df['sobreviveu'].head()

Verificar se existe valores nulos

In [ ]:
df.isnull().any()

Foram encontrados valores nulos nas colunas 'idade', 'cabine' e 'embarcou'

Preenchimento de valores nulos em idade com seu valor médio

In [ ]:
df['idade'].fillna(round(df['idade'].mean()), inplace=True)

Verificar os registros nulos em local de embarque (Embarked)

In [ ]:
df['embarcou'].isnull().sum()

In [ ]:
df[df['embarcou'].isnull()]

Verificar se existe passageiros sem local de embarque marcados como não sobreviventes

In [ ]:
df[df['sobreviveu'] == 'Não']['embarcou'].isnull().any().sum()

Por não existir passageiros sem local de embarque marcado como não sobrevivente todos serão removidos, considerando que estes não estavam a bordo

In [ ]:
df.dropna(subset=['embarcou'], inplace=True)

Validar a remoção de passageiros sem local de embarque

In [ ]:
df['embarcou'].isnull().any().sum()

In [ ]:
df.shape

In [ ]:
df['embarcou'].unique()

Substituição do nome dos locais de embarque abreviados pelo nome completo

In [ ]:
embarked_places = {'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'}
df['embarcou'].replace(embarked_places.keys(), embarked_places.values(), inplace=True)

Categorização das idades no dataframe

In [ ]:
ages = [0, 15, 25, 65, np.inf]
ages_categories = ['Criança', 'Jovem', 'Adulto', 'Idoso']

df['categoria_idade'] = pd.cut(df['idade'], bins=ages, labels=ages_categories,right=True)

Validar o resultado da classificação de idade

In [ ]:
df['categoria_idade'].unique()

# Explorar (Explore)

Matriz de gráficos de dispersão para ideias

In [ ]:
pd.plotting.scatter_matrix(df[['sobreviveu', 'classe', 'idade', 'irmao_conjuge', 'pais_filhos', 'tarifa']], figsize=(15,15));

In [ ]:
df.head(15)

# Configurações e funções

Definir estilo padrão para os gráficos com Seaborn

In [ ]:
sns.set_style('whitegrid')

Definição de função para criar plot figure com título

In [ ]:
def create_figure(title):
    """
        Criar uma figure com titulo posicionado
        
        Args:
            title (String): Valor a ser preenchido no título do gráfico 
            
        Return:
            (plt.Figure) Retorna uma Figure para adicionar gráficos (Plots)
    """
    fig = plt.figure(title)
    fig.suptitle(title, fontsize='x-large', horizontalalignment='center', verticalalignment='bottom')
    plt.subplots_adjust(top=.9)
    plt.tight_layout()
    fig.set_figwidth(10)
    
    return fig


# Perguntas

## 1. Quantos sobreviveram?

Tabela de contagem e proporção de sobreviventes e mortes

In [ ]:
def get_survivors():
    """
        Exibir tabela com contagem de sobreviventes e proporção
        
        Return (dataframe)
    """
    # Definição do dataframe de proporções
    df_survivors = df['sobreviveu'].value_counts().reset_index()
    df_survivors.rename(columns={'index': 'sobreviveu', 'sobreviveu': 'total'}, inplace=True)
    df_survivors['proporcao'] = df_survivors['total'].apply(lambda value: value / df_survivors['total'].sum())
    

    return df_survivors

In [ ]:
df_survivors = get_survivors()
df_survivors[df_survivors['sobreviveu'] == 'Sim']

## 2. Quantas crianças sobreviveram?

In [ ]:
df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts()

49 crianças sobreviveram

## 2. Quantos morreram?

Seleção dos que não sobreviveram no dataframe criado na questão anterior

In [ ]:
df_survivors[df_survivors['sobreviveu'] == 'Não']

## 4. Quantas crianças morreram?

In [ ]:
df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts()

34 crianças morreram

## 5. Qual a idade média entre os sobreviventes?

Seleção dos sobreviventes e cáculo de idade média entre eles

In [ ]:
df[df['sobreviveu'] == 'Sim']['idade'].mean()

## 6. Qual a idade média entre os que não sobreviveram?

In [ ]:
df.query('sobreviveu == "Não"')['idade'].mean()

## 7. Qual a relação dos sobreviventes com a classe de ingresso?

Pesquisa e normalização de sobreviventes por classe

In [ ]:
df.groupby('classe')['sobreviveu'].value_counts(normalize=True)

A classe que apresentou maior número de sobreviventes foi a primeira com 62,6%.

## 8. Qual a relação dos sobreviventes com a classe e sexo?

In [ ]:
df.groupby(['classe','sexo'])['sobreviveu'].value_counts(normalize=True)

As mulheres da primeira classe representaram a maior taxa de sobrevivência, 96,7% e a menor taxa de sobreviventes foi dos homens da terceira classe com apenas 13,5% de sobreviventes.

## 9. Qual ponto de embarque recebeu mais passageiros?

Calcular o volume de embarque para cada local.

In [ ]:
df['embarcou'].value_counts()

Exibir o local com maior frequência

In [ ]:
df['embarcou'].max()

## 10. Qual o sexo mais relevante entre os sobreviventes?

In [ ]:
df.query('sobreviveu == "Sim"')['sexo'].value_counts()

O gênero mais presente entre os sobreviventes é o feminino

## 11. Qual a relação entre as tarifas e os sobreviventes?

In [ ]:
def survivors_fare():
    fig = create_figure('Relação entre tarifas e sobreviventes')
    

In [ ]:
df[df['sobreviveu'] == 'Sim']['tarifa'].describe()

In [ ]:
df[df['sobreviveu'] == 'Não']['tarifa'].describe()

Os sobreviventes apresentam um valor na tarifa um valor médio superior aos que não sobreviveram.

## 12. Quantos bilhetes únicos existem?

In [ ]:
df['ticket'].nunique()

## 13. Quantos bilhetes duplicados existem?

In [ ]:
df['ticket'].duplicated().sum()

## 14. Quais as características dos passageiros com bilhetes duplicados?

In [ ]:
df_duplicated = df[df['ticket'].duplicated(keep=False)]

In [ ]:
df_duplicated.describe(include='all')

Pela descrição dos dados de tickets duplicados é possivel constatar que:
- são no total 342 tickets duplicados,
- destes 342, 177 sobreviveram
- com maior frequência na terceira classe,
- 178 são do sexo feminino
- o ticket identificado como 1601 foi registrado sete vezes
- a cabine mais frequente foi a G6, registrada quatro vezes
- 241 embarcaram em Southampton
- 199 são adultos
- idade média de 26,9 anos

## 15. Qual a relação entre os sobreviventes e os pontos de embarque?

Seleção dos dados de passageiros agrupados por local

In [ ]:
df_surv_embark = df.groupby(['embarcou'])

In [ ]:
df_surv_embark['sobreviveu'].value_counts(normalize=True)

In [ ]:
df_surv_embark['sobreviveu'].value_counts()

In [ ]:
df_surv_embark['classe'].count()

O local de embarque que apresentou a maior número de sobreviventes foi Southampton com 217 (33,6%) sobreviventes do total de 644 que embarcaram, entretanto, Cherbourg apresentou maior percentual de sobreviventes 55,3%, 93 passageiros do total de 168. Queenstown apresentou 38,9% (30) de sobreviventes do total de 77.

# Funções para conclusão e apresentação

# Conclusão

Função para exibir gráfico de sobreviventes

In [ ]:
def plot_survivors():
    """
        Gerar os gráficos de frequência e proporção dos sobreviventes
    """
    title = 'SOBREVIVENTES DO TITANIC\nTotal de passageiros da amostra: {count}'.format(count=df.shape[0])
    
    fig = create_figure(title)
    ax = fig.add_subplot(111)
    #     ax2 = fig.add_subplot(1,2,2)

    # Gerar os gráficos com seaborn
    sns.countplot(data=df, x='sobreviveu', ax=ax, palette='Blues_d')
    
    # Definição dos títulos
    ax.set_xlabel('Sobreviveu')
    ax.set_ylabel('Quantidade')    

plot_survivors()

## 2. Qual a média de idade entre os sobreviventes?

In [ ]:
def survivors_age_mean():
    return df.query('sobreviveu == "Sim"')['idade'].mean()

survivors_age_mean()

Função para plot de densidade de idade de sobreviventes

In [ ]:
def plot_survivors_age_density():
    fig = create_figure('Densidade de idade de sobreviventes')
    ax = fig.add_subplot(111)
    
    sns.violinplot(ax=ax, data=df.query('sobreviveu == "Sim"'), kind='hist', x='sobreviveu', y='idade')
    
plot_survivors_age_density()

# 3. Qual a média de idade entre os que morreram?

In [ ]:
def deads_age_mean():
    return df.query('sobreviveu == "Não"')['idade'].mean()

deads_age_mean()

Função para densidade de idade dos que não sobreviveram

In [ ]:
def plot_deads_age_density():
    fig = create_figure('Densidade de idade de não sobreviventes')
    ax = fig.add_subplot(111)
    sns.violinplot(ax=ax, data=df.query('sobreviveu == "Não"'), kind='hist', x='sobreviveu', y='idade', palette='Reds_d')
    
plot_deads_age_density()

### Sobreviventes por gênero e classe

In [ ]:
# Contagem e proporção de sobreviventes por gênero e classe
df_surv_by_sex = df.groupby(['Pclass', 'Sex', 'Survived']).count()['PassengerId'].reset_index()
df_surv_by_sex.rename(columns={'PassengerId':'Count'}, inplace=True)

def calculate_surv_proportion(row):
    total = df.query('Sex == "{sex}" and Pclass == "{pclass}"'.format(sex = row['Sex'], pclass = row['Pclass']))['Sex'].count()
    return (row['Count'] / total)

df_surv_by_sex['Proportion'] = df_surv_by_sex.apply(lambda row: calculate_surv_proportion(row), axis=1)

df_surv_by_sex.sort_values(['Sex'], ascending=False)

In [ ]:
def segunda_analise():
    ax_by_class = sns.factorplot(data=df_surv_by_sex.sort_values(['Sex'], ascending=False), x='Survived', y='Proportion', hue='Sex', col='Pclass', kind='bar', palette='Blues_d')
    ax_by_class.set_titles('Class {col_name}');

segunda_analise()

In [ ]:
# CONCLUSÃO
conclusions.append((
    'Quais são as características do maior grupo de sobreviventes?',
    'De acordo com o gráfico e os dados de sobreviventes por gênero e classe, as mulheres da primeira classe apresentaram maiores chances, com 96,8% de sobreviventes, seguida das mulheres da segunda classe com proporção de 92,1% e da terceira com 50% de sobreviventes.'
))

### Sobrevivents por local de embarque

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Survived', hue='Sex', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

### Local de embarque por classe e gênero

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Pclass', hue='Sex', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

### Contagem de pessoas por gênero e classe

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Sex', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

### Classificação de idade por classe, local de embarque e sobreviventes

In [ ]:
ax_by_class = sns.factorplot(data=df, x='AgeCategory', hue='Survived', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

In [ ]:
ax_by_class = sns.factorplot(data=df, x='AgeCategory', hue='Sex', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

In [ ]:
def terceira_analise_1():
    ax_by_class = sns.factorplot(data=df, x='AgeCategory', hue='Survived', col='Pclass', kind='count', palette='Blues_d')
    ax_by_class.set_titles('Class {col_name}');

terceira_analise_1()

In [ ]:
def terceira_analise_2():
    # Avaliação de distribuição e densidade de idade por classe e gênero
    fig, ax = plt.subplots(figsize=(12,12))
    sns.violinplot(data=df, ax=ax, x='Pclass', y='Age', col='Pclass', hue='Sex', split=True, inner='quart', palette='Blues_d')
    
    df_age = df['AgeCategory'].value_counts().reset_index()
    df_age['Proportions'] = df['AgeCategory'].value_counts(normalize=True).reset_index()['AgeCategory']
    return df_age    
    
terceira_analise_2()

In [ ]:
def terceira_analise_3():
    return df.groupby('Sex')['Age'].describe()

terceira_analise_3()

In [ ]:
# CONCLUSÃO
conclusions.append((
    'Qual é a descrição de idade dos passageiros?',
    'Para os homens, a idade máxima era de 80 anos, com média em 30 anos e a mínima em 0.4 anos (4,8 meses). No que diz as mulheres, a idade máxima era de 63 anos, média em 28 anos e a mínima em 0.75 ano (9 meses). Em ambos os gêneros, 75% se encaixam em 35 anos, que é possível ver a predominância de pessoas entre 20 e 40 anos no gráfico em violino'
))

In [ ]:
df_surv_by_agecategory = None

def quarta_analise_1():
    df_surv_by_agecategory = df[['Pclass','AgeCategory', 'Survived']].groupby(['Pclass', 'AgeCategory', 'Survived'])['Survived'].agg(['count'])
    return df_surv_by_agecategory
    
df_surv_by_agecategory = quarta_analise_1()
df_surv_by_agecategory

In [ ]:
def quarta_analise_2():
    # Crianças Sobreviventes 
    return df_surv_by_agecategory.query('AgeCategory == "Children"').groupby('Survived').sum()

quarta_analise_2()

In [ ]:
# CONCLUSÃO
conclusions.append((
    'Quantas crianças estavam a bordo? Quantas sobrevieram e quantas morreram?',
    'Haviam 78 crianças a bordo (8,7%) e dessas, 45 sobreviveram e 33 morreram.'
))

### Contagem de familiares (Pais/Filhos) por classe

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Parch', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

### Contagem de irmãos/cônjuges por classe

In [ ]:
ax_by_class = sns.factorplot(data=df, x='SibSp', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

### Avaliação conjunta de famílias (irmãos ou conjuges e pais ou filhos)

In [ ]:
df_parch_sibsp = df.groupby('Pclass')[['Parch', 'SibSp']].agg(['mean', 'count', 'max', 'sum']).reset_index()

df_parch = df.query('Parch > 0')[['Pclass', 'Parch']].groupby('Pclass').count().reset_index()
df_sibsp = df.query('SibSp > 0')[['Pclass', 'SibSp']].groupby('Pclass').count().reset_index()

In [ ]:
# Definir novos indexes
index_columns =[]
for names in df_parch_sibsp.columns.ravel():
    if(names[-1]):
        index_columns.append('{}_{}'.format(names[0], names[-1]))
    else:
        index_columns.append(names[0])

df_parch_sibsp.columns = index_columns

In [ ]:
# Criação de um dataframe com as proporções de passageiros com pais ou filhos e também irmãos ou conjuges.
df_parch_sibsp['Parch_proportion'] = df_parch['Parch'] / df_parch_sibsp['Parch_count']
df_parch_sibsp['SibSp_proportion'] = df_sibsp['SibSp'] / df_parch_sibsp['SibSp_count']

In [ ]:

def quinta_analise_1():
    sns.factorplot(data=df_parch_sibsp, x='Pclass', y='Parch_proportion', kind='bar', palette='Blues_d');
    
    return df_parch_sibsp
    
quinta_analise_1()

In [ ]:
def quinta_analise_2():
    sns.factorplot(data=df_parch_sibsp, x='Pclass', y='SibSp_proportion', kind='bar', palette='Blues_d')
    
quinta_analise_2()

In [ ]:
# CONCLUSÃO
conclusions.append((
        'Em qual classe havia mais familiares?',
        'É possível identificar que a segunda classe apresentou maiores grupos familiares de pais/filhos com 27,1% e a primeira classe na relação de irmãos/conjuges com 36,5%.'
))

## Perguntas e conclusões   

In [ ]:
for index, conclusion in enumerate(conclusions, start=1):
    print('{index}. {pergunta}\n\n\t{resposta}\n\n'.format(index=index, pergunta=conclusion[0], resposta=conclusion[1]))

# TITANIC - Um breve olhar sobre seu naufrágio

Mesmo após um século de seu naufrágio, que ocorreu em 1912, o Titanic é considerado um dos maiores desastres marítimos em tempos de paz. Sua história rendeu livros, filmes e diversos documentarios que buscam explorar e levar informações e curiosidades ao público.

Nesse mesmo sentido, com o objetivo de aplicar conhecimentos do curso de Data Science do Udacity, este projeto irá explorar um conjunto de dados disponibilizado na etapa final através do [link](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59e4fe3d_titanic-data-6/titanic-data-6.csv).


# Perguntas

Perguntas elaboradas ao conhecer o conjunto de dados fornecido:

## 1. Quais são os números de sobreviventes?
    
A primeira pergunta buscou conhecer o tamanho da tragédia. Quantas pessoas sobreviveram, quantas vidas foram perdidas, qual o tamanho da amostra. Nesse sentido busquei a descrição do conjunto de dados e a exibição de grafícos de frequência e proporção para sobreviventes e mortes.

## 2. Quais são as características do maior grupo de sobreviventes?
    
Com o propósito de conhecer melhor sobre os sobreviventes e mortos os dividi em grupos de gênero, classe, local de embarque, com o propósito de encontrar qual grupo apresentava maior taxa de sobrevivência. Explorando suas descrições, normalizações e gráficos de frequência.

## 3. Qual é a descrição de idade dos passageiros?
    
Dentre a amostra fornecida, qual é a idade? Como poderia classifica-los, quais eram as faixas etárias, qual a idade mais presente entre eles? Qual era a idade da pessoa mais velha, e a mais nova? Para isso, os dados foram agrupados por classe, faixa etária e gênero e posteriormente explora-los.

## 4. Quantas crianças estavam a bordo? Quantas sobrevieram e quantas morreram?
    
Crianças, quantas eram? Sobreviveram? Qual é o balanço? Utilizando também da classificação de faixa etaria, foi possível mapear as crianças, idades e o balanço de sobreviventes.

## 5. Em qual classe haviam mais familiares? (Irmãos, conjuges, pais, filhos)
    
Após explorar os dados sobre as crianças, busquei relacionar quais dados poderiam relacionar famílias a bordo, irmãos ou conjuges e pais ou filhos. Contando e normalizando informações sobre as pessoas que apresentavam ao menos um relacionamento familiar nos campos disponíveis.




# Resumo

## 1. Quais são os números de sobreviventes?

In [ ]:
primeira_analise()

In [ ]:
print(conclusions[0][1])

## 2. Quais são as características do maior grupo de sobreviventes?

In [ ]:
segunda_analise()

In [ ]:
print(conclusions[1][1])

## 3. Qual é a descrição de idade dos passageiros?

In [ ]:
terceira_analise_1()

In [ ]:
terceira_analise_2()

In [ ]:
terceira_analise_3()

In [ ]:
print(conclusions[2][1])

## 4. Quantas crianças estavam a bordo? Quantas sobrevieram e quantas morreram?

In [ ]:
quarta_analise_1()

In [ ]:
quarta_analise_2()

In [ ]:
print(conclusions[3][1])

## 5. Em qual classe haviam mais familiares? (Irmãos, conjuges, pais, filhos)¶

In [ ]:
quinta_analise_1()

In [ ]:
quinta_analise_2()

In [ ]:
display(Markdown(conclusions[4][1]))


# Limpeza de dados

O primeiro passo foi listar os tipos de dados utilizados e se havia campos nulos. Feito isso, foi constatado que haviam campos nulos em Idade (Age), Local de embarcação (Embarked) e Identificação da cabine (Cabin). 
Assim:
- Os campos com idade nula foram preenchidos com o valor da média;
- Foi feita uma pesquisa pelos passageiros que não possuiam local de embaração e consultado seu status de sobrevivente ou não e foi constatado que nenhum havia morrido. Nesse sentido, foi considerado de que não estavam a bordo e seus registros removidos;
- Cabin foi mantido nulo, pois diante do quadro de informações disponíveis, não apresentava potencial informativo, poderia ser descartado;
- As abreviações dos locais de embaração foram substituidos pelo nome completo;
- Foram criadas categorias para faixa etária;


# Consultas e referências

- [Seaborn API documentation](https://seaborn.pydata.org/api.html)
- [Stack Overflow: Pandas](https://stackoverflow.com/questions/tagged/pandas)
- [Stack Overflow: Seaborn](https://stackoverflow.com/questions/33524694/plotting-with-seaborn)
